In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

format_dict = {
               'price':'{:.2f}','buy_p':'{:.2f}','sell_p':'{:.2f}','diff':'{:.2f}',
               'ppu':'{:.4f}',
               'net':'{:,.2f}','fee':'{:,.2f}','vat':'{:,.2f}',
               'buy_amt':'{:,.2f}','sell_amt':'{:,.2f}','profit':'{:,.2f}',
               'div_amt':'{:,.2f}','ttl_amt':'{:,.2f}','pct':'{:,.2f}',
               'qty':'{:,}','ttl_qty':'{:,}','number':'{:,}','days':'{:,}'
              }
pd.set_option("display.max_rows", None)

cols = 'name ttl_qty buy_amt ppu div_amt buy_date x_date p_date days pct year quarter'.split()
colt = 'name ttl_qty buy_amt ppu div_amt buy_date x_date p_date days pct year quarter number'.split()

name = 'MCS'

### 1ST Batch

In [2]:
div_year = 2016
div_quarter = 4
ids = '284,286,287,288,289,290,901'

In [3]:
sql = '''
SELECT T.name, SUM(qty) AS ttl_qty, 
SUM(B.net) AS buy_amt, MIN(date) AS buy_date
FROM buys B
JOIN stocks T ON B.stock_id = T.id 
WHERE B.id IN (%s)
GROUP BY T.name'''
sql = sql % (ids)
buys = pd.read_sql(sql, conpf)
buys.style.format(format_dict)

,name,ttl_qty,buy_amt,buy_date
0,MCS,"70,000.0","1,171,589.23",2016-08-26


In [4]:
sql = """
SELECT id, name, year, quarter, number, ppu, net AS div_amt, x_date, p_date
FROM dividends
WHERE name = '%s'
AND year >= %s
ORDER BY name, year DESC, quarter DESC
"""
sql = sql % (name, div_year)
dividends = pd.read_sql(sql, conpf)
dividends.style.format(format_dict)

,id,name,year,quarter,number,ppu,div_amt,x_date,p_date
0,211,MCS,2020,4,"75,000",0.6000,"43,050.00",2021-03-15,2021-04-28
1,18,MCS,2020,2,"75,000",0.4000,"29,325.00",2020-08-27,2020-09-10
2,20,MCS,2019,4,"75,000",0.4000,"29,250.00",2020-04-29,2020-04-29
3,34,MCS,2019,3,"75,000",0.2500,"18,525.00",2019-11-21,2019-12-04
4,71,MCS,2018,4,"70,000",0.5000,"34,370.00",2019-03-12,2019-04-29
5,123,MCS,2017,4,"80,000",0.4000,"29,840.00",2018-04-18,2018-05-07
6,134,MCS,2017,2,"80,000",0.2000,"14,400.00",2017-08-24,2017-09-11
7,155,MCS,2016,4,"80,000",0.4800,"37,760.00",2017-04-19,2017-05-09


In [5]:
df_merge = pd.merge(buys, dividends, on='name', how='inner')
df_merge['div_amt'] = df_merge.ppu * df_merge.ttl_qty
df_merge['days'] = df_merge.p_date - df_merge.buy_date
df_merge['days'] = df_merge['days']/np.timedelta64(1,'D')
df_merge['days'] = df_merge['days'].astype(int)
df_merge['pct'] = round(df_merge['div_amt']/df_merge['buy_amt']*36500/df_merge['days'],2)
df_merge[cols]

,name,ttl_qty,buy_amt,ppu,div_amt,buy_date,x_date,p_date,days,pct,year,quarter
0,MCS,70000.0,1171589.23,0.60,42000.0,2016-08-26,2021-03-15,2021-04-28,1706,0.77,2020,4
1,MCS,70000.0,1171589.23,0.40,28000.0,2016-08-26,2020-08-27,2020-09-10,1476,0.59,2020,2
2,MCS,70000.0,1171589.23,0.40,28000.0,2016-08-26,2020-04-29,2020-04-29,1342,0.65,2019,4
3,MCS,70000.0,1171589.23,0.25,17500.0,2016-08-26,2019-11-21,2019-12-04,1195,0.46,2019,3
4,MCS,70000.0,1171589.23,0.50,35000.0,2016-08-26,2019-03-12,2019-04-29,976,1.12,2018,4
5,MCS,70000.0,1171589.23,0.40,28000.0,2016-08-26,2018-04-18,2018-05-07,619,1.41,2017,4
6,MCS,70000.0,1171589.23,0.20,14000.0,2016-08-26,2017-08-24,2017-09-11,381,1.14,2017,2
7,MCS,70000.0,1171589.23,0.48,33600.0,2016-08-26,2017-04-19,2017-05-09,256,4.09,2016,4


In [6]:
df_grp =df_merge.groupby(['name'])['div_amt'].sum()
div1 = df_grp
div1

name
MCS    226100.0
Name: div_amt, dtype: float64

In [7]:
file_name = 'div-' + name + '.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
data_file, output_file, box_file

('../data/div-MCS.csv',
 '\\Users\\User\\iCloudDrive\\div-MCS.csv',
 '\\Users\\User\\Dropbox\\div-MCS.csv')

In [8]:
df_merge[colt].to_csv(output_file)
df_merge[colt].to_csv(data_file)
df_merge[colt].to_csv(box_file)

### 2nd Batch

In [9]:
div_year = 2019
div_quarter = 3
ids = '653'

In [10]:
sql = '''
SELECT T.name, SUM(qty) AS ttl_qty, 
SUM(B.net) AS buy_amt, MIN(date) AS buy_date
FROM buys B
JOIN stocks T ON B.stock_id = T.id 
WHERE B.id IN (%s)
GROUP BY T.name'''
sql = sql % (ids)
buys = pd.read_sql(sql, conpf)
buys

,name,ttl_qty,buy_amt,buy_date
0,MCS,5000.0,83684.95,2019-11-11


In [11]:
sql = """
SELECT id, name, year, quarter, number, ppu, net AS div_amt, x_date, p_date
FROM dividends
WHERE name = '%s'
AND year >= %s
ORDER BY name, year DESC, quarter DESC
"""
sql = sql % (name, div_year)
dividends = pd.read_sql(sql, conpf)
dividends.style.format(format_dict)

,id,name,year,quarter,number,ppu,div_amt,x_date,p_date
0,211,MCS,2020,4,"75,000",0.6000,"43,050.00",2021-03-15,2021-04-28
1,18,MCS,2020,2,"75,000",0.4000,"29,325.00",2020-08-27,2020-09-10
2,20,MCS,2019,4,"75,000",0.4000,"29,250.00",2020-04-29,2020-04-29
3,34,MCS,2019,3,"75,000",0.2500,"18,525.00",2019-11-21,2019-12-04


In [12]:
df_merge = pd.merge(buys, dividends, on='name', how='inner')
df_merge['div_amt'] = df_merge.ppu * df_merge.ttl_qty
df_merge['days'] = df_merge.p_date - df_merge.buy_date
df_merge['days'] = df_merge['days']/np.timedelta64(1,'D')
df_merge['days'] = df_merge['days'].astype(int)
df_merge['pct'] = round(df_merge['div_amt']/df_merge['buy_amt']*36500/df_merge['days'],2)
df_merge[cols]

,name,ttl_qty,buy_amt,ppu,div_amt,buy_date,x_date,p_date,days,pct,year,quarter
0,MCS,5000.0,83684.95,0.60,3000.0,2019-11-11,2021-03-15,2021-04-28,534,2.45,2020,4
1,MCS,5000.0,83684.95,0.40,2000.0,2019-11-11,2020-08-27,2020-09-10,304,2.87,2020,2
2,MCS,5000.0,83684.95,0.40,2000.0,2019-11-11,2020-04-29,2020-04-29,170,5.13,2019,4
3,MCS,5000.0,83684.95,0.25,1250.0,2019-11-11,2019-11-21,2019-12-04,23,23.70,2019,3


In [13]:
df_grp =df_merge.groupby(['name'])['div_amt'].sum()
div2 = df_grp
div2

name
MCS    8250.0
Name: div_amt, dtype: float64

In [14]:
df_merge[colt].to_csv(output_file,mode='a',header=False)
df_merge[colt].to_csv(data_file,mode='a',header=False)
df_merge[colt].to_csv(box_file,mode='a',header=False)

### End of process

In [15]:
div_year = 2016
div_quarter = 4

In [16]:
sql = """
SELECT id, name, year, quarter, number, ppu, number * ppu AS div_amt, x_date, p_date
FROM dividends
WHERE name = '%s'
AND ((year = %s AND quarter >= %s) 
OR (year > %s))
ORDER BY name, year, quarter
"""
sql = sql % (name, div_year, div_quarter, div_year)
dividends = pd.read_sql(sql, conpf)
dividends.style.format(format_dict)

,id,name,year,quarter,number,ppu,div_amt,x_date,p_date
0,155,MCS,2016,4,"80,000",0.4800,"38,400.00",2017-04-19,2017-05-09
1,134,MCS,2017,2,"80,000",0.2000,"16,000.00",2017-08-24,2017-09-11
2,123,MCS,2017,4,"80,000",0.4000,"32,000.00",2018-04-18,2018-05-07
3,71,MCS,2018,4,"70,000",0.5000,"35,000.00",2019-03-12,2019-04-29
4,34,MCS,2019,3,"75,000",0.2500,"18,750.00",2019-11-21,2019-12-04
5,20,MCS,2019,4,"75,000",0.4000,"30,000.00",2020-04-29,2020-04-29
6,18,MCS,2020,2,"75,000",0.4000,"30,000.00",2020-08-27,2020-09-10
7,211,MCS,2020,4,"75,000",0.6000,"45,000.00",2021-03-15,2021-04-28


In [17]:
dividends.div_amt.sum()

245150.0